In [4]:
from dotenv import load_dotenv
import os
load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
HUGGINGFACEHUB_API_TOKEN = os.getenv('HUGGINGFACEHUB_API_TOKEN')


In [19]:
from langchain.llms import OpenAI
llm = OpenAI(openai_api_key=OPENAI_API_KEY,temperature=0.6)
text="Who is the author of the book Harry Potter?"
print(llm.predict(text))



J.K. Rowling


In [21]:
from langchain import HuggingFaceHub
llm_huggingface=HuggingFaceHub(huggingfacehub_api_token = HUGGINGFACEHUB_API_TOKEN, repo_id="google/flan-t5-large",model_kwargs={"temperature":0.4,"max_length":264})
output=llm_huggingface.predict("Write a haiku about sadness")
print(output)

sadness is a sour taste


In [22]:
text="Write a haiku about sadness"
print(llm.predict(text))



Tears fall like raindrops
Heavy heart weighs like boulders
Sadness consumes me

